In [1]:
# Selenium with Google Colab

# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# # Set options to be headless, ..
# from selenium import webdriver
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# browser = webdriver.Chrome('chromedriver',options=options)

In [2]:
import pandas as pd
import csv
import time
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys

In [3]:
# # Khoi tao file diemthi_nghean.csv va dat ten cot
# with open('./diemthi_nghean.csv', 'w') as file:
#     writer = csv.writer(file)
#     writer.writerow(['So bao danh', 'Toan', 'Ngu Van', 'Ngoai Ngu', 'Vat Ly', 'Hoa Hoc', 'Sinh Hoc', 'Lich Su', 'Dia Ly', 'GDCD'])

In [4]:
# Ham tra ve dong cuoi cua file csv
def return_last_line(filepath):    
    with open(filepath, 'r') as file:        
        for x in file:
            pass
        return x   
# Ham luu data vao file csv
def save_data(filepath, data):
    with open(filepath, 'a') as file: 
        writer = csv.writer(file)
        writer.writerow(data)

# Ham dien vao o va enter
def filled(box_location, filled_str):
    box_location.send_keys(str(filled_str))
    box_location.send_keys(Keys.ENTER)

# Ham xoa ki tu trong o 
def delete(box_location, character_num):
    for i in range(character_num):
        box_location.send_keys(Keys.BACKSPACE)

# Ham crawl data
def crawler(begin, end):
    # Khai bao bien temporary data
    temp = [0] * 10
    # Lap tu SBD begin den end
    for id_num in list(range(begin, end)):
        filled(search_field, id_num)
        if id_num == begin:
            sleep(1)
        sleep(0.3) # Loading

        # Kiem tra SBD co ton tai hay khong
        if browser.find_elements_by_xpath('/html/body/div[1]/div/section/div/div/div[3]/div/div/div/div[1]/div/div/div/div[1]') == []:
            # Xoa SBD va kiem tra lai mot lan nua roi thoat khoi lan lap nay
            delete(search_field, len(str(id_num)))
            filled(search_field, id_num)
            sleep(0.5) # Loading
            if browser.find_elements_by_xpath('/html/body/div[1]/div/section/div/div/div[3]/div/div/div/div[1]/div/div/div/div[1]') == []:
                temp = [id_num] + [''] * 9
                save_data(filepath, temp)
                delete(search_field, len(str(id_num)))
                sleep(0.1) # Loading 
            continue

        # Kiem tra SBD == id_num, neu khong bang thi load lai cho den khi bang
        SBD = browser.find_element_by_css_selector('#nav-home > div > div.info-student > p:nth-child(2)').text[-8:]
        while(str(id_num) != SBD):
            delete(search_field, len(str(id_num)))
            filled(search_field, id_num)
            sleep(0.2) # Loading
            SBD = browser.find_element_by_css_selector('#nav-home > div > div.info-student > p:nth-child(2)').text[-8:]
        
        # Luu data vao temp
        temp[0] = id_num
        for selector_num in list(range(1, 10)):
            selector = '#nav-home > div > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child({value})'
            selector = selector.format(value = str(selector_num))
            data = browser.find_element_by_css_selector(selector)
            temp[selector_num] = data.text.strip()
        # Luu data tu temp vao file csv 
        save_data(filepath, temp)
        print(','.join(list(map(lambda x: str(x), temp))) + ' done!!!')
        # Xoa tim kiem
        delete(search_field, len(str(id_num)))

In [5]:
# Khai bao browser
browser = webdriver.Chrome(executable_path='./chromedriver')

# CSV filepath
filepath = './diemthi_nghean.csv'

# Mo url cua trang diem thi 
url = 'https://diemthi.tuoitre.vn/thpt-2021'
browser.get(url)

# Tim o dien so bao danh
search_field = browser.find_element_by_name('identifyNumber')

# Tim begin = SBD cuoi cung trong file csv + 1
last_line = return_last_line(filepath)
if last_line[0:2] != '29':
    begin = 29000001
else:
    begin = int(last_line[0:8]) + 1

# Do duoc end = 29034379
end = 29034481

# Crawling
crawler(begin, end)

# Dong trinh duyet
browser.close()


29034480,7.20,6.25,4.20,,,,2.75,5.50,4.00 done!!!
